In [4]:
from selenium import webdriver

import pandas as pd

from datetime import datetime

from selenium.common.exceptions import StaleElementReferenceException

from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By

# Spécifiez le chemin du pilote ChromeDriver
chrome_driver_path = 'C:\\Program Files (x86)\\chromedriver.exe'  # Assurez-vous d'échapper correctement les caractères '\'

# Initialisez le service du navigateur Chrome avec le chemin du pilote
chrome_service = webdriver.chrome.service.Service(chrome_driver_path)

# Initialisez les options du navigateur Chrome
chrome_options = webdriver.ChromeOptions()

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
import requests

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [5]:
def get_description_text(product_url):
    # Envoyer une requête GET pour récupérer le contenu HTML de la page
    
    driver.get(product_url)
    #response = requests.get(product_url)

    # Vérifier si la requête a réussi (statut 200)
    #if response.status_code == 200:
        
    # Utiliser BeautifulSoup pour analyser le contenu HTML
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    #soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver la balise 'h2' avec le texte "Description"
    description_heading = soup.find('h2', text='Description')

    # Vérifier si la balise 'h2' a été trouvée
    if description_heading:
        # Trouver la balise 'p' suivante après la balise 'h2'
        description_paragraph = description_heading.find_next('p')

        # Récupérer le texte de la balise 'p'
        description_text = description_paragraph.get_text(strip=True) if description_paragraph else None

        # Retourner le texte de la description
        return description_text
    else:
        return "Pas de Caracteristique"
#else:
    #return f"Échec de la requête. Statut : {response.status_code}"

In [ ]:
Category_Links = [
    
    'https://www.adjovan.com/product-category/fruits-legumes/',
    'https://www.adjovan.com/product-category/viande-volaille/',
    'https://www.adjovan.com/product-category/articles-pour-bebe/',
    'ttps://www.adjovan.com/product-category/boulangerie-patisserie/',
    'https://www.adjovan.com/product-category/charcuterie/',
    'https://www.adjovan.com/product-category/croustilles-collations/',
    'https://www.adjovan.com/product-category/epices-sauces-huiles/',
    'https://www.adjovan.com/product-category/sante-et-bien-etre/',
    'https://www.adjovan.com/product-category/poissons/',
    'https://www.adjovan.com/product-category/produits-laitiers-et-oeuf/',
    'https://www.adjovan.com/product-category/produits-menagers/',
    'https://www.adjovan.com/product-category/hygiene-beaute/'    
]


data = []


for category_url in Category_Links :

    # Utilisation de Selenium avec le navigateur Chrome
    driver = webdriver.Chrome()
    driver.get(category_url)

    # Attendre quelques secondes pour permettre le chargement complet de la page
    driver.implicitly_wait(5)

    while True:
        
        
        # Récupérer le contenu de la page après le chargement dynamique
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extraire les informations des produits sur la page actuelle
        products = soup.find_all('li', class_='product')

        for product in products:
            # Extraire le titre du produit
            title = product.find('h3', class_='product-name').text.strip()

            # Extraire le prix du produit
            #price = product.find('span', class_='price').text.strip()
            
            # Extraire le prix du produit s'il est disponible
            price_element = product.find('span', class_='price')
            price = price_element.text.strip() if price_element else "Prix non disponible"
            
            quantity = title.split('[')[-1].split(']')[0]
            
            # Obtenir la date de collecte actuelle
            date_de_collecte = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            product_link = product.find('div', class_='image-block').find('a')['href']
        
            caracteristique = get_description_text(product_link)

             # Ajouter les informations à la liste de données
            data.append({
            
              "Date_de_collecte": date_de_collecte,  # Vous devrez attribuer une date
              "Code_site":"adjovan",
              "Libelle_du_produit": title,
              "Quantite": quantity,
              "Prix_du_produit": price
              "Caracteristique" : caracteristique
                
               })
    
        try:
            
            #On repart sur la page initiale
            driver.get(category_url)
            driver.implicitly_wait(5)
            page_source = driver.page_source
            
            # Trouver le lien vers la page suivante (pagination)
            next_page_link = driver.find_element(By.CSS_SELECTOR, 'a.next.page-numbers')
            #next_page_link = driver.find_element_by_css_selector('a.next.page-numbers')

            # Vérifier s'il y a une page suivante
            if 'disabled' in next_page_link.get_attribute('class'):
        
                break  # Sortir de la boucle s'il n'y a pas de page suivante

            # Cliquer sur le lien vers la page suivante
            #next_page_link.click()
            
             # Utiliser execute_script pour cliquer sur le bouton "Next"
            driver.execute_script("arguments[0].click();", next_page_link)

            # Attendre quelques secondes pour permettre le chargement complet de la page suivante
            driver.implicitly_wait(5)
        
        except NoSuchElementException:
        
            print("Dernière page atteinte.")
            break
            
# Créer un DataFrame à partir des données
df = pd.DataFrame(data)
df['Unite'] = df['Quantite'].str.extract('([A-Za-z]+)')

# Afficher le DataFrame
len(df)

There was an error managing chrome (request or response body error: operation timed out); using driver found in the cache


Dernière page atteinte.


There was an error managing chrome (request or response body error: operation timed out); using driver found in the cache


Dernière page atteinte.


There was an error managing chrome (request or response body error: error reading a body from connection: stream error received: not a result of an error); using driver found in the cache


Dernière page atteinte.
Dernière page atteinte.


In [53]:
df.head()

,Date_de_collecte,Code_site,Libelle_du_produit,Quantite,Prix_du_produit,Caracteristique,Unite
0,2024-01-26 16:16:03,adjovan,AIL [500G],500G,900CFA,Pas de Caracteristique,G
1,2024-01-26 16:16:08,adjovan,ANANAS [UNITÉ],UNITÉ,450CFA 350CFA,"L’ananas est riche en vitamines, enzymes et an...",UNIT
2,2024-01-26 16:16:09,adjovan,ANETH [BOTTE],BOTTE,300CFA,Une botte de 100g minimum ultra fraîche!,BOTTE
3,2024-01-26 16:16:11,adjovan,AUBERGINE ”PETIT PETIT’ [500G],500G,500CFA,Pas de Caracteristique,G
4,2024-01-26 16:16:12,adjovan,AUBERGINE BLANCHE [500G],500G,400CFA,Pas de Caracteristique,G


In [ ]:
Category_Links = [
    
    'https://www.adjovan.com/product-category/fruits-legumes/',
    'https://www.adjovan.com/product-category/viande-volaille/',
    'https://www.adjovan.com/product-category/articles-pour-bebe/',
    'ttps://www.adjovan.com/product-category/boulangerie-patisserie/',
    'https://www.adjovan.com/product-category/charcuterie/',
    'https://www.adjovan.com/product-category/croustilles-collations/',
    'https://www.adjovan.com/product-category/epices-sauces-huiles/',
    'https://www.adjovan.com/product-category/sante-et-bien-etre/',
    'https://www.adjovan.com/product-category/poissons/',
    'https://www.adjovan.com/product-category/produits-laitiers-et-oeuf/',
    'https://www.adjovan.com/product-category/produits-menagers/',
    'https://www.adjovan.com/product-category/hygiene-beaute/'    
]

data = []

def get_description_text(product_url):
    # Envoyer une requête GET pour récupérer le contenu HTML de la page
    
    driver.get(product_url)
    #response = requests.get(product_url)

    # Vérifier si la requête a réussi (statut 200)
    #if response.status_code == 200:
        
    # Utiliser BeautifulSoup pour analyser le contenu HTML
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    #soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver la balise 'h2' avec le texte "Description"
    description_heading = soup.find('h2', text='Description')

    # Vérifier si la balise 'h2' a été trouvée
    if description_heading:
        # Trouver la balise 'p' suivante après la balise 'h2'
        description_paragraph = description_heading.find_next('p')

        # Récupérer le texte de la balise 'p'
        description_text = description_paragraph.get_text(strip=True) if description_paragraph else None

        # Retourner le texte de la description
        return description_text
    else:
        return "Pas de Caracteristique"
#else:
    #return f"Échec de la requête. Statut : {response.status_code}"

for category_url in Category_Links :

    # Utilisation de Selenium avec le navigateur Chrome
    driver = webdriver.Chrome()
    driver.get(category_url)

    # Attendre quelques secondes pour permettre le chargement complet de la page
    driver.implicitly_wait(5)

    while True:
        # Récupérer le contenu de la page après le chargement dynamique
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extraire les informations des produits sur la page actuelle
        products = soup.find_all('li', class_='product')

        for product in products:
            # Extraire le titre du produit
            title = product.find('h3', class_='product-name').text.strip()

            # Extraire le prix du produit
            #price = product.find('span', class_='price').text.strip()
            
            # Extraire le prix du produit s'il est disponible
            price_element = product.find('span', class_='price')
            price = price_element.text.strip() if price_element else "Prix non disponible"
            
            quantity = title.split('[')[-1].split(']')[0]
            
            product_link = product.find('div', class_='image-block').find('a')['href']
            
            print(product_link)
            
            description = get_description_text(product_link)
            
            # Obtenir la date de collecte actuelle
            date_de_collecte = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            print(description)

        try:
            
            # Trouver le lien vers la page suivante (pagination)
            next_page_link = driver.find_element(By.CSS_SELECTOR, 'a.next.page-numbers')
            #next_page_link = driver.find_element_by_css_selector('a.next.page-numbers')

            # Vérifier s'il y a une page suivante
            if 'disabled' in next_page_link.get_attribute('class'):
        
                break  # Sortir de la boucle s'il n'y a pas de page suivante

            # Cliquer sur le lien vers la page suivante
            #next_page_link.click()
            
             # Utiliser execute_script pour cliquer sur le bouton "Next"
            driver.execute_script("arguments[0].click();", next_page_link)

            # Attendre quelques secondes pour permettre le chargement complet de la page suivante
            driver.implicitly_wait(5)
        
        except NoSuchElementException:
        
            print("Dernière page atteinte.")
            break
            
# Créer un DataFrame à partir des données
#df = pd.DataFrame(data)

# Afficher le DataFrame
#print(df)

https://www.adjovan.com/shop/en-gros/fruits-gros/carton-de-mangues-10kg/


C:\Users\dell\AppData\Local\Temp\ipykernel_6864\2915143690.py:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  description_heading = soup.find('h2', text='Description')


C’est la saison, Profitez!
https://www.adjovan.com/shop/fruits-legumes/feuilles/celeri-1-botte/
Fort en goût, le céleri permet de multiples déclinaisons en cuisine.
https://www.adjovan.com/shop/epices-sauces-huiles/sel-epices/champignons-de-paris-frais-500g/
VARIÉTÉ: PIED COUPE PLAT
https://www.adjovan.com/shop/epices-sauces-huiles/sel-epices/champignon-local-100g/
Pas de Caracteristique
https://www.adjovan.com/shop/epices-sauces-huiles/sel-epices/champignons-noirs-duy-anh-100g/
La plupart des gens peuvent manger le champignon noir (PLEUROTE), il est approprié surtout aux patients qui souffrent des maladies cardio-vasculaires et cérébro-vasculaires, ou de la lithiase. La consommation de champignon noir est particulièrement conseillée à ceux qui ont une carence en fer, aux mineurs, aux métallurgistes, aux travailleurs du textile, et aux barbiers.
https://www.adjovan.com/shop/fruits-legumes/produits-secs/champignons-noirs-25g/
Pas de Caracteristique
https://www.adjovan.com/shop/fruits-le

Pour poursuivre l’initiation de bébé, vous pouvez maintenant introduire la viande avec notre recette Purée Carottes Jambon. Pour varier les plaisirs, Blédina propose plus de 15 petits pots aux recettes toutes simples !
https://www.adjovan.com/shop/articles-pour-bebe/aliments-pour-bebe/compotes-desserts/bledina-petits-pots-poires-2x130g-4-6mois/
Avec une texture lisse, mon 1er petit pot est idéal pour les premières cuillères et offre tout le goût de la poire dans une recette simple.
https://www.adjovan.com/shop/articles-pour-bebe/aliments-pour-bebe/compotes-desserts/bledina-petits-pots-pomme-framboise-4x130g-des-6mois/
Avec ce petit pot, bébé découvre le goût de la framboise associée à la pomme dans une recette simple. Les fruits blédina sont spécialement cultivés pour les tout-petits, selon des règles très strictes. Ils sont soigneusement cueillis au rythme des saisons !
https://www.adjovan.com/shop/articles-pour-bebe/aliments-pour-bebe/repas/bledina-petits-pots-pommes-2x130g-4-6mois/


Offrez à vos enfants un délicieux biscuit au blé complet et au bon goût vanille pour le goûter.
https://www.adjovan.com/shop/croustilles-collations/biscuits/biscuits-casse-croute-bn-original-400g/
Cecasse croûteest idéal pour le petit déjeuner, très proche du Petit Lu, il fait des merveilles lorsqu’il est plongé dans un bol de thé ou de café.
Dernière page atteinte.
https://www.adjovan.com/shop/epices-sauces-huiles/huile-et-vinaigre/bidon-huile-aya-20l/
Unehuilelégère et digeste.Ayaest unehuilede cuisson, assaisonnement et friture saine pour la santé cardiaque et des repas savoureux.
https://www.adjovan.com/shop/epices-sauces-huiles/huile-et-vinaigre/bidon-huile-dinor-20-l/
Catégorie : Huile de table raffinée.
https://www.adjovan.com/shop/epices-sauces-huiles/sel-epices/cancancan-nora-30g/
Épicetrès prisée des amateurs de viandes braisées, le CanCanCan est un mélange très aimé en Afrique Subsaharienne.
https://www.adjovan.com/shop/epices-sauces-huiles/sel-epices/cannelle-moulue-nora-35

In [4]:
 # Ajouter les informations à la liste de données
            data.append({
            
              "date_de_collecte": date_de_collecte,  # Vous devrez attribuer une date
              "Libelle_du_produit": title,
              "Quantite": quantity,
              "Prix_du_produit": price,
              "Description": description
                
               })

,date_de_collecte,Libelle_du_produit,Quantite,Prix_du_produit
0,2024-01-24 17:54:31,AIL [500G],500G,900CFA
1,2024-01-24 17:54:31,ANANAS [UNITÉ],UNITÉ,450CFA 350CFA
2,2024-01-24 17:54:31,ANETH [BOTTE],BOTTE,300CFA
3,2024-01-24 17:54:31,AUBERGINE ”PETIT PETIT’ [500G],500G,500CFA
4,2024-01-24 17:54:31,AUBERGINE BLANCHE [500G],500G,400CFA
